In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import numpy as np
import re
import time
import requests
import xmltodict
import ast 


In [ ]:
df = pd.read_csv('asylum_cases_structure.csv')

In [ ]:
DETAILED_LINK_URL = "https://data.rechtspraak.nl/uitspraken/content?id={}"

def get_ecli_from_detailed_link(detailed_link):
    try:
        return detailed_link.split("=")[-1]
    except:
        return None


def get_former_relatie_from_called_api(called_api_page):
    
    doc = xmltodict.parse(called_api_page.text)#     dict_instantie = doc["open-rechtspraak"]["rdf:RDF"]["rdf:Description"][0]["psi:Instantie"]
    try:
        dict_instantie = doc['open-rechtspraak']['rdf:RDF']['rdf:Description'][0]['dcterms:relation']['@ecli:resourceIdentifier']
    except TypeError:
        
        return 'multiple'
    
    return dict_instantie

def check_multiple_zaaknummers(case):
    
    ecli_temp = get_ecli_from_detailed_link(case)
    detailed_sentence = requests.get(
        DETAILED_LINK_URL.format(ecli_temp))
    
    zaaknummer_temp = get_zaaknummer_from_called_api(detailed_sentence)
        
    if 'en' in zaaknummer_temp or ',' in zaaknummer_temp:
        return True
    
    return False


def get_former_relatie(ecli_code):
    
    print(ecli_code)
        
    time.sleep(0.0001)
    
    ecli_temp = get_ecli_from_detailed_link(ecli_code)
    
    detailed_sentence = requests.get(
            DETAILED_LINK_URL.format(ecli_temp), timeout = 15)
    
    try:
        kenmerken = get_former_relatie_from_called_api(detailed_sentence)
    
    except IndexError as i:
        
        kenmerken = "no former relation"  
        
    except KeyError:
        
        kenmerken = "no former relation" 
        
    except requests.exceptions.ReadTimeout as err:
        kenmerken = "no former relation" 
    
    return kenmerken


In [ ]:
df['former_relatie'] = df.case.apply(get_former_relatie)

In [ ]:
df.to_csv('cases_with_relatie.csv')